In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import tarfile
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from multiprocessing import Process# this is used for multithreading
import multiprocessing
import codecs# this is used for file operations 
import random as r
import time
import math
import cv2
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import log_loss,confusion_matrix
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,MaxPooling2D,Conv2D,Dropout,BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
# from keras.utils import np_utils
from keras.preprocessing import image
from sklearn.datasets import load_files
import cv2
import pickle
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,precision_score,recall_score
from statistics import mode
from sklearn.utils.multiclass import unique_labels

In [2]:
label_path='/kaggle/input/labels-data/labels/'
# This function will create an index file with image paths and labels in csv format
# obtained csv will have two columns path(path to acess the image),label(class of the image)
def create_csv(file_path):
    images=[]
    labels=[]
    label_file=pd.DataFrame(columns=['path','label'],index=None)
    file=open(file_path)
    for ele in file:
        images.append(ele.split(' ')[0])
        labels.append(ele.split(' ')[1].rstrip())
    label_file.path=images
    label_file.label=labels
    return label_file

train_label_path=label_path+'train.txt' 
test_label_path=label_path+'test.txt' 
val_label_path=label_path+'val.txt'
# creating index file for train,test and val dataset
train_label=create_csv(train_label_path)
test_label=create_csv(test_label_path)
val_label=create_csv(val_label_path)


In [15]:
val_label

,path,label
0,imagesg/g/t/h/gth35e00/2024525661.tif,11
1,imagesi/i/y/k/iyk38c00/512015827+-5827.tif,0
2,imagesr/r/r/e/rre21e00/87103403.tif,0
3,imagesk/k/s/u/ksu44c00/03636607.tif,4
4,imagesr/r/a/i/rai09d00/50437856-7857.tif,14
...,...,...
39995,imageso/o/u/k/ouk93f00/0013006838.tif,10
39996,imagesf/f/f/b/ffb52c00/2074103881.tif,11
39997,imagesg/g/h/b/ghb11f00/0001251052.tif,15
39998,imagesl/l/c/k/lck71f00/2016003416.tif,9


In [3]:
# denifing the metadatas
train_dir='train'
test_dir='test'
cv_dir='val'
classes = {'0': 'letter',
 '1': 'form',
 '10': 'budget',
 '11': 'invoice',
 '12': 'presentation',
 '13': 'questionnaire',
 '14': 'resume',
 '15': 'memo',
 '2': 'email',
 '3': 'handwritten',
 '4': 'advertisement',
 '5': 'scientific report',
 '6': 'scientific publication',
 '7': 'specification',
 '8': 'file folder',
 '9': 'news article'}

In [9]:
# Creating train,validation and Test directories
# os.mkdir('train')
os.mkdir('test')
os.mkdir('val')

In [10]:
# run this cell to enable intra domain transfer learning
regions = ['Whole','header', 'footer', 'left_body', 'right_body']
for region in regions:
#     os.mkdir('train/'+region)
    os.mkdir('val/'+region)
classes_name=[v for (k,v) in classes.items()]
for region in regions:
    for label in classes_name:
#         os.mkdir('train/'+region+'/'+label)
        os.mkdir('val/'+region+'/'+label)

In [25]:
shutil.rmtree('/kaggle/working/train', ignore_errors=True)

In [11]:
from tqdm import tqdm
import os
import shutil
import pandas as pd

# Assuming 'label' column in train_label contains the label information
# classes is a dictionary that maps label to the destination folder
# For example, classes = {'label1': 'folder1', 'label2': 'folder2', ...}

# Create a new DataFrame to store the count of images for each label
label_counts = train_label['label'].value_counts()

# Set the maximum number of images per label
max_images_per_label = 1000

for label, count in tqdm(label_counts.items()):
    # Limit the number of images to move for each label
    num_images_to_move = min(count, max_images_per_label)

    # Filter the DataFrame for the current label and select the first `num_images_to_move` images
    images_to_move = train_label[train_label['label'] == label].head(num_images_to_move)

    for i in range(len(images_to_move)):
        source = '/kaggle/input/rvlcdip/images/' + images_to_move.iloc[i]['path']
        dest = 'train/Whole/' + classes[str(label)]

        if not os.path.exists(dest):
            os.makedirs(dest)

        dest_file = os.path.join(dest, images_to_move.iloc[i]['path'].split('/')[-1])

        if not os.path.exists(dest_file):
            shutil.copy(source, dest)


16it [01:58,  7.39s/it]


In [13]:
from tqdm import tqdm
import os
import shutil
import pandas as pd

# Assuming 'label' column in train_label contains the label information
# classes is a dictionary that maps label to the destination folder
# For example, classes = {'label1': 'folder1', 'label2': 'folder2', ...}

# Create a new DataFrame to store the count of images for each label
label_counts = val_label['label'].value_counts()

# Set the maximum number of images per label
max_images_per_label = 200

for label, count in tqdm(label_counts.items()):
    # Limit the number of images to move for each label
    num_images_to_move = min(count, max_images_per_label)

    # Filter the DataFrame for the current label and select the first `num_images_to_move` images
    images_to_move = val_label[val_label['label'] == label].head(num_images_to_move)

    for i in range(len(images_to_move)):
        source = '/kaggle/input/rvlcdip/images/' + images_to_move.iloc[i]['path']
        dest = 'val/Whole/' + classes[str(label)]

        if not os.path.exists(dest):
            os.makedirs(dest)

        dest_file = os.path.join(dest, images_to_move.iloc[i]['path'].split('/')[-1])

        if not os.path.exists(dest_file):
            shutil.copy(source, dest)

16it [00:00, 35.23it/s]


In [14]:
val_dir='val/Whole'
count,limit=0,2000
for folder in tqdm(os.listdir(val_dir)):
  #os.shuffle(val_dir+'/'+folder)
  for image in os.listdir(val_dir+'/'+folder):
        img = cv2.imread(val_dir+'/'+folder+'/'+image,0)
        height = img.shape[0]
        width = img.shape[1]
        header = img[0:(int(height*0.33)), 0:width]
        footer = img[int(height*0.67):height, 0:width]
        left_body = img[int(height*0.33):int(height*0.67), 0:int(width*0.5)]
        right_body = img[int(height*0.33):int(height*0.67), int(width*0.5):width]
        header_path='val/header/'+folder+'/'+image.split('/')[-1]
        footer_path='val/footer/'+folder+'/'+image.split('/')[-1]
        left_path='val/left_body/'+folder+'/'+image.split('/')[-1]
        right_path='val/right_body/'+folder+'/'+image.split('/')[-1]
        if(os.path.exists(header_path)==False):
            cv2.imwrite(header_path, header)
        if(os.path.exists(footer_path)==False):
            cv2.imwrite(footer_path, footer)
        if(os.path.exists(left_path)==False):
            cv2.imwrite(left_path, left_body)
        if(os.path.exists(right_path)==False):
            cv2.imwrite(right_path, right_body)
        count+=1
        if(count==limit):
            break

100%|██████████| 16/16 [00:25<00:00,  1.57s/it]


In [15]:
# Training configuaration for CNN- to be trained on whole images
img_width, img_height = 224,224
train_dir = "train/Whole"
val_dir = "val/Whole"
test_dir="test"
nb_train_samples = 16000
nb_validation_samples = 3200 
batch_size = 32
epochs = 10
num_classes=16
log=[]

In [16]:
# preparing Training and Validation data using ImageDataGenerator
train_datagen = ImageDataGenerator(
rescale = 1./255,
featurewise_std_normalization=True)

val_datagen = ImageDataGenerator(
rescale = 1./255,
featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
train_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

val_generator = val_datagen.flow_from_directory(
val_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

Found 16000 images belonging to 16 classes.
Found 3200 images belonging to 16 classes.


In [21]:
# Importing keras InceptionRsnetv2 pretrained model (on ImageNet)
# We will use the ImageNet weight to initialize the model and fine tune using backpropagation(Transfer Learning) 
from keras import regularizers
model=applications.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
for layer in model.layers[:15]:
    layer.trainable = False
#Adding custom Layers 
x = model.output
x=Dropout(0.5)(x)
x = Flatten()(x)
output = Dense(num_classes, activation="softmax")(x)
model = Model(inputs = model.input, outputs = output)

In [22]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

checkpoint = ModelCheckpoint("inceptionresnet.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=0.01, mode='max')
callbacks = [checkpoint, reduce_lr]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=callbacks,
    verbose=1
)

log.append(history)

Epoch 1/10


2023-11-12 15:36:43.880122: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


500/500 [==============================] - ETA: 0s - loss: 2.2941 - accuracy: 0.4957
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to inceptionresnet.h5
500/500 [==============================] - 256s 346ms/step - loss: 2.2941 - accuracy: 0.4957 - val_loss: 6575.1992 - val_accuracy: 0.4000 - lr: 0.0010
Epoch 2/10
500/500 [==============================] - ETA: 0s - loss: 2.1780 - accuracy: 0.5815
Epoch 2: val_accuracy improved from 0.40000 to 0.59000, saving model to inceptionresnet.h5
500/500 [==============================] - 170s 340ms/step - loss: 2.1780 - accuracy: 0.5815 - val_loss: 6.3018 - val_accuracy: 0.5900 - lr: 0.0010
Epoch 3/10
500/500 [==============================] - ETA: 0s - loss: 2.3873 - accuracy: 0.6101
Epoch 3: val_accuracy did not improve from 0.59000
500/500 [==============================] - 169s 338ms/step - loss: 2.3873 - accuracy: 0.6101 - val_loss: 454.0276 - val_accuracy: 0.4997 - lr: 0.0010
Epoch 4/10
500/500 [========================

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=callbacks,
    verbose=1
)

log.append(history)

In [38]:
# predictions
from keras.models import load_model
from keras.preprocessing import image

img = image.load_img('/kaggle/input/test-image/50643700-3700.tif', target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array.reshape((1, 224, 224, 3))
img_array /= 255.0

prediction = model.predict(img_array)
predicted_class_index = int(np.argmax(prediction, axis=-1))
predicted_class_name = classes.get(str(predicted_class_index), 'unknown')
print("Predicted Folder: ", predicted_class_name)

1/1 [==============================] - 0s 36ms/step
Predicted Folder:  presentation
